In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import polars as pl

from datetime import datetime
import time

import utils.source

from utils.request_market_data import (
    request_prices_to_landing,
    write_prices_to_raw,
    read_prices_from_lake,
    trasnform_trusted_prices
)

from utils.request_cvm_ciasabertas import (
    download_files,
    read_files_from_landing,
    write_files_to_lake,
    read_files_from_lake,
    trasnform_trusted_accounts,
    trasnform_trusted_tickers,
    transform_trusted_stocks,
    trasnform_refined_accounts
)


In [2]:
(
    read_files_from_lake(zone='raw', table='itr/dre')
    .filter(pl.col('CNPJ_CIA')=='33.000.167/0001-01')
    .with_columns(YEAR=pl.col('DT_REFER').dt.year())
    .pivot(
        index=['YEAR'],
        columns='CD_CONTA',
        values='VL_CONTA',
        aggregate_function='count'
    )
    .sort('YEAR')
    .write_csv("/home/glauco/follow-the-market/outputs/output.csv")
)

In [3]:
DOCUMENT = 'fre'
REPORT = 'capital_social'
DIR = DOCUMENT+'/'+REPORT

In [4]:
df = read_files_from_lake('raw', DIR)

In [5]:
df = transform_trusted_stocks(df)

In [6]:
write_files_to_lake(df, 'trusted', DIR)

In [ ]:




if DOCUMENT == 'itr':
    df = trasnform_trusted_accounts(df)
elif DOCUMENT == 'fca':
    df = trasnform_trusted_tickers(df)

write_files_to_lake(df, 'trusted', DIR)